___
# README: Processing strata files from Kostelka et al's paper

- This notebook translates the strata code in *CV_syntax.do* into Python language.
- Before running the next cells, please upload the following .dta files to the Files section in Google Colab (left panel)
  - Q_G_for_merge.dta
  - V_Dem_v10_for_merge.dta
  - Comp_voting_new.dta
  - Sanctions.dta
  - HansonSigman_source_extract.dta
- The .dta files can be found [here](https://drive.google.com/drive/folders/1eKftI7R1_FqgFvqYCz2nP0sw8Islf5St?usp=drive_link).
- **Remark:** Although our work uses data from [International IDEA](https://www.idea.int/data-tools/data/voter-turnout-database) website, we replicate the dataset from Kostelka et al's paper for comparative analysis.

___

## 1. Data processing

In [ ]:
import pandas as pd
pd.set_option("display.max_columns", None)

PATH_DATA = "/content"

# Uploading datasets
df_qg = pd.read_stata(f"{PATH_DATA}/Q_G_for_merge.dta")
df_vdem = pd.read_stata(f"{PATH_DATA}/V_Dem_v10_for_merge.dta")
df_turnout = pd.read_stata(f"{PATH_DATA}/Comp_voting_new.dta")
df_sanctions = pd.read_stata(f"{PATH_DATA}/Sanctions.dta")
df_hanson_sigman = pd.read_stata(f"{PATH_DATA}/HansonSigman_source_extract.dta")

# Basic processing
df_qg.drop(columns=["cname"], inplace=True)
df_sanctions.drop(columns=["EL_TYPE"], inplace=True)
df_sanctions.drop(columns=["multiple_el_id"], inplace=True)

df_vdem["year"] = df_vdem["year"].astype("int")
df_turnout["year"] = df_turnout["year"].astype("int")
df_hanson_sigman["year"] = df_hanson_sigman["year"].astype("int")

# Initial dataset
df_turnout = df_turnout.merge(df_qg, left_on=["PLT_ccode","year"], right_on=["country", "year"], how="left")
df_turnout.rename(columns={"country_y": "country"}, inplace=True)

# Adding Quality of Government, V-dem, state capacity (Hanson and Sigman 2021), and sanctions
df_turnout = df_turnout.merge(df_vdem, on=["country", "year"], how="left")
df_turnout = df_turnout.merge(df_hanson_sigman, on=["country", "year"], how="left")
df_turnout = df_turnout.merge(df_sanctions, on=["country", "year"], how="left")

# Additional columns
df_turnout["fine_GDP"] = df_turnout["monetaryfine_USD_2011"] / (df_turnout["gdp_pc"] / 1000)
df_turnout["max_fine_GDP"] = df_turnout["maximummonetaryfine_USD_2011"] / (df_turnout["gdp_pc"] / 1000)

# Coding the sanctions variable as zero for countries that do not enforce CV
df_turnout.loc[df_turnout["CV_enforced"] != 0, ["maximummonetaryfine_salary"]] *= df_turnout["CV_enforced"]
df_turnout.loc[df_turnout["CV_enforced"] != 0, ["monetaryfine_salary"]] *= df_turnout["CV_enforced"]
df_turnout.loc[df_turnout["CV_enforced"] != 0, ["fine_GDP"]] *= df_turnout["CV_enforced"]
df_turnout.loc[df_turnout["CV_enforced"] != 0, ["max_fine_GDP"]] *= df_turnout["CV_enforced"]
df_turnout.loc[df_turnout["CV_enforced"] != 0, ["non_monetary_sanct"]] *= df_turnout["CV_enforced"]

df_turnout.loc[df_turnout["CV_enforced"] == 0, ["maximummonetaryfine_salary"]] = 0
df_turnout.loc[df_turnout["CV_enforced"] == 0, ["monetaryfine_salary"]] = 0
df_turnout.loc[df_turnout["CV_enforced"] == 0, ["fine_GDP"]] = 0
df_turnout.loc[df_turnout["CV_enforced"] == 0, ["max_fine_GDP"]] = 0
df_turnout.loc[df_turnout["CV_enforced"] == 0, ["non_monetary_sanct"]] = 0

# Setting the TSCS structure
df_turnout.sort_values(by=["country", "year", "EL_TYPE", "multiple_el_id"], inplace=True)
df_turnout["el_num"] = df_turnout.groupby("country").cumcount() + 1
df_turnout.set_index(["country", "el_num"], inplace=True)

# Combining variables Electoral System and Election Type
df_turnout["el_system"] = None
df_turnout.loc[df_turnout["EL_TYPE"] == 1, "el_system"] = "Presidential Election"
df_turnout.loc[df_turnout["EL_TYPE"] == 0, "el_system"] = "v2elparlel"

df_turnout.reset_index(inplace=True)

In [ ]:
# Completing missing values using the literature
df_turnout.loc[(df_turnout["country"] == 10) & (df_turnout["el_system"].isna()) & (df_turnout["EL_TYPE"] == 0) & (df_turnout["year"] == 2006), "el_system"] = "El. System: Mixed"
df_turnout.loc[(df_turnout["country"] == 2) & (df_turnout["el_system"].isna()) & (df_turnout["EL_TYPE"] == 0), "el_system"] = "El. System: Proportional"
df_turnout.loc[(df_turnout["country"] == 21) & (df_turnout["year"] == 1946)  & (df_turnout["EL_TYPE"] == 0), "el_system"] = "El. System: Majoritarian"
df_turnout.loc[(df_turnout["country"] == 29) & (df_turnout["year"] == 1997) & (df_turnout["EL_TYPE"] == 0), "el_system"] = "El. System: Proportional"
df_turnout.loc[(df_turnout["country"] == 105) & (df_turnout["year"].isin([1965,1968])) & (df_turnout["EL_TYPE"] == 0), "el_system"] = "El. System: Majoritarian"

# Creating year since 1945
df_turnout["y_since_1945"] = df_turnout["year"] - 1944

# Creating a version of unsanctioned CV in which sanctioned CV is not nested
df_turnout["CV_2"] = df_turnout["CV"]
df_turnout.loc[df_turnout["CV_enforced"] != 0, "CV_2"] = 0
df_turnout.rename(columns={"CV_2": "CV Unsanctioned", "CV_enforced": "CV Sanctioned"}, inplace=True)

# Generating turnout as a proportion
df_turnout["turnout_as_proportion"] = df_turnout["Turnout"] / 100

df_turnout.rename(columns={"CV Unsanctioned": "CV_Unsanctioned", "CV Sanctioned": "CV_Sanctioned"}, inplace=True)

In [ ]:
print(df_turnout.shape)
df_turnout.head(3)

(1421, 102)


,country,el_num,country_x,year,Turnout,EL_TYPE,Legislative1stround,Presidential1stround,PLT_cyear,PLT_ccode,PLT_scode,PLT_polity,PLT_polity2,PLT_COUNTRY,PLT_dem6,PLT_dem0,PLT_dem6_time,PLT_dem0_time,PLT_beg_dem0_spell,PLT_spell_dem0,PLT_beg_dem6_spell,PLT_spell_dem6,country_to_merge,v2ddthreci,v2ddthreor,v2ddthrepl,v2ddthrerf,WB_countryname,WB_countrycode,WB_v_1136,WB_v_1193,WB_v_1199,WB_v_1205,WB_v_1298,WB_v_1299,WB_v_1301,WB_v_1303,WB_v_1351,WB_v_1359,WB_v_1417,WB_v_1476,WB_v_1552,WB_v_400,WB_v_513,WB_v_515,WB_v_530,WB_v_570,WB_v_753,WB_v_777,multiple_el_id,Decisiveness,Closeness,Concurrent,CV,CV_Sanctioned,Voting_age,Direct_pres,electorate_size,Last_Election,Date,share_new,gdp_pc,GDP_1960_start_year,growth_1y,growth_2y,growth_3y,growth_4y,voting_age18_cum,starting_year_18,after_1959_share,decade_1940s,decade_1950s,decade_1960s,decade_1970s,decade_1980s,decade_1990s,decade_2000s,decade_2010s,beg_dem,postcommunist,new,pre_1974_new,postcommunist_new,post_1974_new,log_gdp_pc,log_el_size,wdi_mortnn,wdi_taxrev,v2elparlel,e_regionpol,Capacity,non_monetary_sanct,monetaryfine_USD_2011,maximummonetaryfine_USD_2011,monetaryfine_salary,maximummonetaryfine_salary,fine_GDP,max_fine_GDP,el_system,y_since_1945,CV_Unsanctioned,turnout_as_proportion
0,2.0,1.0,United States,1946,38.799999,Legislative,1,0,21946,2,USA,10,10,United States,1,1,138,147,0,1,0,2,117.0,0.0,0.0,0.0,0.0,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,3.5,8.5,0.0,0.0,0.0,21.0,1.0,88.400002,0.0,d_05_11,0.0,14471.0,1941.0,-9.511005,-6.067883,-2.035702,0.691272,0.0,1972.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1799.0,0.0,0.0,0.0,0.0,0.0,9.579902,4.481872,NaN,NaN,NaN,Latin America,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,None,2,0.0,0.388
1,2.0,2.0,United States,1948,52.200001,Legislative,1,1,21948,2,USA,10,10,United States,1,1,140,149,0,1,0,2,117.0,0.0,0.0,0.0,0.0,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,2.6,7.2,1.0,0.0,0.0,21.0,1.0,95.300003,0.0,d_02_11,0.0,14559.0,1941.0,3.571175,0.303596,-3.080852,-3.893284,0.0,1972.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1799.0,0.0,0.0,0.0,0.0,0.0,9.585964,4.557030,NaN,NaN,NaN,Latin America,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,None,4,0.0,0.522
2,2.0,3.0,United States,1948,52.200001,Presidential,1,1,21948,2,USA,10,10,United States,1,1,140,149,0,1,0,2,117.0,0.0,0.0,0.0,0.0,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.4,4.5,1.0,0.0,0.0,21.0,1.0,95.300003,0.0,d_02_11,0.0,14559.0,1941.0,3.571175,0.303596,-3.080852,-3.893284,0.0,1972.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1799.0,0.0,0.0,0.0,0.0,0.0,9.585964,4.557030,NaN,NaN,NaN,Latin America,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,None,4,0.0,0.522


## 2. Basic descriptives

In [ ]:
# Number of unique countries
# ==========================

len(df_turnout["PLT_scode"].unique())

116

In [ ]:
# Years in panel
# ==============

df_turnout["year"].min(), df_turnout["year"].max()

(1945, 2017)

In [ ]:
# Number of countries with CV in dataset
# ======================================

temp_1 = df_turnout[df_turnout["CV"]==1]["PLT_scode"]
temp_2 = df_turnout[df_turnout["CV_Sanctioned"]==1]["PLT_scode"]

temp_3 = pd.concat([temp_1, temp_2], axis=0)
len(temp_3.unique())

25

In [ ]:
# Identifying changes in CV status per country
# ============================================

df_turnout["cv_change"] = (
                        df_turnout
                        .sort_values(['PLT_COUNTRY','EL_TYPE','year'], ascending=[True,True,True])
                        .groupby(["PLT_COUNTRY","EL_TYPE"])["CV"]
                        .transform(lambda x: x.diff())
                      )

df_turnout["cv_enf_change"] = (
                            df_turnout
                            .sort_values(['PLT_COUNTRY', 'EL_TYPE', 'year'], ascending=[True,True,True])
                            .groupby(["PLT_COUNTRY","EL_TYPE"])["CV_Sanctioned"]
                            .transform(lambda x: x.diff())
                          )

df_turnout["turnout_change"] = (
                        df_turnout
                        .sort_values(['PLT_COUNTRY','EL_TYPE','year'], ascending=[True,True,True])
                        .groupby(["PLT_COUNTRY","EL_TYPE"])["Turnout"]
                        .transform(lambda x: x.diff())
                      )

In [ ]:
# List of countries that adopt or abandon CV
# ==========================================

df_turnout[(df_turnout["cv_change"].isin([1,-1])) | (df_turnout["cv_enf_change"].isin([1,-1]))]

,country,el_num,country_x,year,Turnout,EL_TYPE,Legislative1stround,Presidential1stround,PLT_cyear,PLT_ccode,PLT_scode,PLT_polity,PLT_polity2,PLT_COUNTRY,PLT_dem6,PLT_dem0,PLT_dem6_time,PLT_dem0_time,PLT_beg_dem0_spell,PLT_spell_dem0,PLT_beg_dem6_spell,PLT_spell_dem6,country_to_merge,v2ddthreci,v2ddthreor,v2ddthrepl,v2ddthrerf,WB_countryname,WB_countrycode,WB_v_1136,WB_v_1193,WB_v_1199,WB_v_1205,WB_v_1298,WB_v_1299,WB_v_1301,WB_v_1303,WB_v_1351,WB_v_1359,WB_v_1417,WB_v_1476,WB_v_1552,WB_v_400,WB_v_513,WB_v_515,WB_v_530,WB_v_570,WB_v_753,WB_v_777,multiple_el_id,Decisiveness,Closeness,Concurrent,CV,CV_Sanctioned,Voting_age,Direct_pres,electorate_size,Last_Election,Date,share_new,gdp_pc,GDP_1960_start_year,growth_1y,growth_2y,growth_3y,growth_4y,voting_age18_cum,starting_year_18,after_1959_share,decade_1940s,decade_1950s,decade_1960s,decade_1970s,decade_1980s,decade_1990s,decade_2000s,decade_2010s,beg_dem,postcommunist,new,pre_1974_new,postcommunist_new,post_1974_new,log_gdp_pc,log_el_size,wdi_mortnn,wdi_taxrev,v2elparlel,e_regionpol,Capacity,non_monetary_sanct,monetaryfine_USD_2011,maximummonetaryfine_USD_2011,monetaryfine_salary,maximummonetaryfine_salary,fine_GDP,max_fine_GDP,el_system,y_since_1945,CV_Unsanctioned,turnout_as_proportion,cv_change,cv_enf_change,turnout_change
150,91.0,8.0,Honduras,2005,45.970001,Legislative,1,1,912005,91,HON,7,7,Honduras,1,1,17,26,0,18,0,4,44.0,0.0,0.0,0.0,0.0,Honduras,HND,18.664064,65.898919,NaN,7373430.0,29.141809,NaN,NaN,NaN,NaN,NaN,2.5,NaN,48.516,NaN,6.214406,0.002616,1799.490332,NaN,111890.0,NaN,1.0,1.562500,5.468750,1.0,1.0,0.0,18.0,1.0,3.900000,NaN,,0.000000,4123.0,NaN,6.952010,5.334058,4.499720,5.462000,0.702663,1981.0,0.749323,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1979.0,0.0,0.0,0.0,0.0,0.0,8.324336,1.360977,8.6,17.468057,NaN,E. Europe and C. Asia,1.117,0.0,NaN,NaN,0.0,0.0,0.0,0.0,None,61,1.0,0.4597,0.0,-1.0,-20.629997
151,91.0,9.0,Honduras,2005,55.080002,Presidential,1,1,912005,91,HON,7,7,Honduras,1,1,17,26,0,18,0,4,44.0,0.0,0.0,0.0,0.0,Honduras,HND,18.664064,65.898919,NaN,7373430.0,29.141809,NaN,NaN,NaN,NaN,NaN,2.5,NaN,48.516,NaN,6.214406,0.002616,1799.490332,NaN,111890.0,NaN,1.0,4.400000,3.400000,1.0,1.0,0.0,18.0,1.0,3.900000,NaN,,0.000000,4123.0,NaN,6.952010,5.334058,4.499720,5.462000,0.702663,1981.0,0.749323,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1979.0,0.0,0.0,0.0,0.0,0.0,8.324336,1.360977,8.6,17.468057,NaN,E. Europe and C. Asia,1.117,0.0,NaN,NaN,0.0,0.0,0.0,0.0,None,61,1.0,0.5508,0.0,-1.0,-11.519997
460,NaN,NaN,Netherlands,1971,79.099998,Legislative,1,0,2101971,210,NTH,10,10,Netherlands,1,1,27,27,0,4,0,4,78.0,0.0,0.0,0.1,0.0,Netherlands,NLD,0.461553,390.832257,NaN,13194497.0,29.697130,14.512520,NaN,NaN,75.386787,19.667931,NaN,NaN,61.913,16.299999,1.710736,2.997305,25120.536641,6.41001,33760.0,NaN,1.0,25.400000,2.800000,0.0,0.0,0.0,21.0,0.0,8.100000,NaN,,0.000000,18347.0,1964.0,4.756195,5.419790,6.118378,8.319354,0.000000,1972.0,0.000000,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1944.0,0.0,0.0,0.0,0.0,0.0,9.817222,2.091864,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,0.0,0.0,0.0,0.0,None,27,0.0,0.7910,-1.0,-1.0,-15.800003
555,NaN,NaN,Cyprus,1981,95.699997,Legislative,1,0,3521981,352,CYP,10,10,Cyprus,1,1,14,22,0,1,0,3,23.0,0.0,0.0,0.0,0.0,Cyprus,CYP,3.978470,74.585823,NaN,689173.0,22.207479,16.118860,NaN,NaN,67.935493,2.868380,NaN,NaN,60.809,NaN,3.751129,NaN,13273.645980,3.79132,9240.0,NaN,1.0,17.200001,0.900000,0.0,1.0,1.0,18.0,1.0,0.300000,NaN,,0.000000,13131.0,1979.0,-0.942969,1.578768,5.235718,8.762892,0.000000,1998.0,0.107299,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1959.0,0.0,0.0,0.0,0.0,0.0,9.482731,-1.203973,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,37,0.0,0.9570,1.0,1.0,10.399994
583,NaN,NaN,Turkey,1983,92.300003,Legislative,1,0,6401983,640,TUR,7,7,Turkey,1,1,1,1,1,8,1,8,113.0,0.0,0.1,1.0,0.0,Turkey,TUR,2.453052,61.163705,NaN,47073422.0,28.554529,18.515619,NaN,8169.0,38.910030,6.293110,NaN,NaN,48.812,NaN,0.074581,NaN,5310.341937,2.64760,769630.0,NaN,1.0,4.900000,14.600000,0.0,1.0,0.0,21.0,0.0,19.800001,NaN,

In [11]:
# Number of uniques countries that adopt or abandon CV
# ====================================================

filt_analysis = (df_turnout["cv_change"].isin([1,-1])) | (df_turnout["cv_enf_change"].isin([1,-1]))

len(df_turnout[filt_analysis]["PLT_COUNTRY"].unique())

11

## 3. Save processed dataset

In [ ]:
select_cols = ["year","PLT_COUNTRY","EL_TYPE","CV","CV_Unsanctioned","CV_Sanctioned","Turnout"]

df_turnout.loc[:, select_cols].to_csv("proc-kostelka.csv", index=False)